In [1]:
!python -V

Python 3.11.9


In [3]:
import evidently
print(evidently.__version__)

0.4.28


In [21]:
import requests
import datetime
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric, ColumnQuantileMetric

from joblib import load, dump
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

pd.set_option('display.float_format', lambda x: '%.2f' % x)

import warnings
warnings.filterwarnings('ignore')

In [17]:
file = ('green_tripdata_2024-03.parquet')
path = ('./data')

print("Download files: ")


url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
request = requests.get(url, stream=True)
save_path = f"{path}/{file}"
with open(save_path, 'wb') as handle:
    for data in tqdm(request.iter_content(), 
                     desc=f"{file}",
                     postfix= f"save to {save_path}",
                     total=int(request.headers.get('content-length', 0))):
        handle.write(data)

Download files: 


green_tripdata_2024-03.parquet: 100%|██████████| 1372372/1372372 [00:13<00:00, 100900.70it/s, save to ./data/green_tripdata_2024-03.parquet]


In [18]:
march_data = pd.read_parquet(f'./data/{file}')

In [19]:
march_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2024-03-01 00:10:52,2024-03-01 00:26:12,N,1.00,129,226,1.00,1.72,12.80,1.00,0.50,3.06,0.00,NaN,1.00,18.36,1.00,1.00,0.00
1,2,2024-03-01 00:22:21,2024-03-01 00:35:15,N,1.00,130,218,1.00,3.25,17.70,1.00,0.50,0.00,0.00,NaN,1.00,20.20,2.00,1.00,0.00
2,2,2024-03-01 00:45:27,2024-03-01 01:04:32,N,1.00,255,107,2.00,4.58,23.30,1.00,0.50,3.50,0.00,NaN,1.00,32.05,1.00,1.00,2.75
3,1,2024-03-01 00:02:00,2024-03-01 00:23:45,N,1.00,181,71,1.00,0.00,22.50,0.00,1.50,0.00,0.00,NaN,1.00,24.00,1.00,1.00,0.00
4,2,2024-03-01 00:16:45,2024-03-01 00:23:25,N,1.00,95,135,1.00,1.15,8.60,1.00,0.50,1.00,0.00,NaN,1.00,12.10,1.00,1.00,0.00


In [20]:
march_data.shape

(57457, 20)